In [207]:
import cv2
cv2.__version__

'4.5.1'

In [208]:
#DETEKSI OBJEK DHULU
#program deteksi objek

import matplotlib.pyplot as plt
import cv2
import numpy as np
import math

box=[]

# Load Yolo
net = cv2.dnn.readNet("yolov4-obj_best.weights","yolov4-obj.cfg")
#net = cv2.dnn.readNet("20_yolov3.weights","20_yolov3.cfg")
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

color_blue = (255,0,0)
color_red = (0,0,255)
#colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Loading image
image = cv2.imread("../data/CNR/samples/cam8.jpg") 
#img = cv2.resize(imges, None, fx=1, fy=1)
height, width, channels = image.shape

# Detecting objects
#blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
blob = cv2.dnn.blobFromImage(image, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.45:
            #if class_id == 2:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.45, 0.4)

#print(indexes)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        
        box.append(boxes[i])
        #print(boxes[i])
        label = str(classes[class_ids[i]])
        #color = colors[i]
        cv2.rectangle(image, (x, y), (x + w, y + h), color_blue, 2)
        cv2.putText(image, label, (x, y + 30), font, 2, color_blue, 2)
        cv2.putText(image, str("{:.2f}".format(confidences[i])), (x, y + 50), font, 2, color_red, 2)
        
cv2.imshow("output1",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [209]:
#Load koodinat npy
ROI_slot = np.load('../data/CNR/coordinate/cam8.npy',allow_pickle=True)

In [210]:
boxess = []
for i in range(len(indexes)):
    boxess.append([])
    boxess[i].append(box[i][0])
    boxess[i].append(box[i][1])
    boxess[i].append(box[i][2]+box[i][0])
    boxess[i].append(box[i][3]+box[i][1])

In [211]:
boxess

[[125, 35, 172, 85],
 [373, 109, 433, 171],
 [289, 38, 345, 87],
 [396, 38, 448, 89]]

In [212]:
ROI_car = []
for i in range(len(indexes)):
    ROI_car.append([(boxess[i][0],boxess[i][3]), (boxess[i][0],boxess[i][1]), (boxess[i][2],boxess[i][1]), (boxess[i][2],boxess[i][3])])    

In [213]:
#inisialisasi array
diagonal_slot = []                      
centroid_slot_x = []
centroid_slot_y = []
for i in range (len(ROI_slot)):            
    new = []
    new1 = []
    new2 = []
    new3 = [0,0]
    for j in (ROI_slot[i]):
        new.append(0)
        new1.append(0)
        new2.append(0)
    diagonal_slot.append(new)    
    centroid_slot_x.append(new1)
    centroid_slot_y.append(new2)    
    
#print(centroid_slot)
#print(diagonal_slot)
area_slot = []
for i in range (len(ROI_slot)):            
    new = []
    for j in (ROI_slot[i]):
        new.append(0)
    area_slot.append(new)
    
for row in range(len(ROI_slot)):
    for slots in range(len(ROI_slot[row])):
        x1 = ROI_slot[row][slots][0][0]
        y1 = ROI_slot[row][slots][0][1]
        x2 = ROI_slot[row][slots][1][0]
        y2 = ROI_slot[row][slots][1][1]
        x3 = ROI_slot[row][slots][2][0]
        y3 = ROI_slot[row][slots][2][1]
        x4 = ROI_slot[row][slots][3][0]
        y4 = ROI_slot[row][slots][3][1]
        
        #menghitung centroid slot        
        centroid_slot_x[row][slots] = (x3+x1)/2
        centroid_slot_y[row][slots] = (y3+y1)/2        
        
        #print((x3+x1)/2)
        #menghitung diagonal slot
        d1 = math.sqrt(((y3-y1)**2)+((x3-x1)**2))
        d2 = math.sqrt(((y4-y2)**2)+((x4-x2)**2))
        if d1<d2:
            diagonal_slot[row][slots] = d1
        else:
            diagonal_slot[row][slots] = d2
            
        area_slot[row][slots] = (abs(((x1*y2-y1*x2) + (x2*y3-y2*x3) + (x3*y4-y3*x4) + (x4*y1-y4*x1))/2))    

In [214]:
import numpy as np

arr = np.array(area_slot)
#print(arr)

<ipython-input-214-68a3b4d53db3>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr = np.array(area_slot)


In [215]:
#Export ke npy
from numpy import save

save("areaCNR/area8.npy",arr)

In [216]:
print(centroid_slot_x)

[[964.0, 910.0, 857.5, 805.5, 752.5, 694.5, 640.0, 589.0, 537.5, 477.5, 421.0, 366.0, 318.0, 259.5, 205.0, 148.5, 92.5, 42.0], [961.5, 901.5, 833.5, 762.0, 694.0, 617.0, 545.5, 473.0, 401.5, 324.0, 257.0, 183.5, 112.0, 47.5], [960.0, 891.5, 811.0, 724.0, 639.5, 547.0, 465.0, 382.5, 300.5, 205.5, 119.5, 47.0], [947.5, 856.0, 730.5, 593.5, 471.0, 315.5, 188.5, 76.5], [150.5, 480.0, 826.0]]


In [217]:
#Load images
image = cv2.imread("../data/CNR/samples/cam8.jpg")
image = cv2.resize(image, (image.shape[1],image.shape[0]), interpolation = cv2.INTER_AREA)
plottedImage = image.copy()

In [218]:
#Menghitung ED, kemudian IoU bila ED mobil < th
from shapely.geometry import Polygon

ED = []
perc_ED = []
temp = []
IoU = []
Status = []
for i in range (len(ROI_slot)):            
    new = []
    new1 = []
    new2 = []
    for j in (ROI_slot[i]):
        new.append(0)
        new1.append(0)
        new2.append(0)
    ED.append(new)
    perc_ED.append(new)
    IoU.append(new2)
    Status.append(new1)
    
intersect=[]
count=0
    
for row in range(len(ROI_slot)):
    for slots in range(len(ROI_slot[row])):
        centroid_slot_xx = centroid_slot_x[row][slots]
        centroid_slot_yy = centroid_slot_y[row][slots]
        temp = []
        intersect=[]
        for car in range(len(ROI_car)):
            x1 = ROI_car[car][0][0]
            y1 = ROI_car[car][0][1]
            x3 = ROI_car[car][2][0]
            y3 = ROI_car[car][2][1]
            
            #print(x1)
            
            centroid_car_x = (x1+x3)/2
            centroid_car_y = (y1+y3)/2
            temp.append(math.sqrt(((centroid_slot_yy-centroid_car_y)**2)+((centroid_slot_xx-centroid_car_x)**2)))
            p = Polygon(ROI_slot[row][slots])
            q = Polygon(ROI_car[car])
            intersect.append(q.intersection(p).area)
        
        min_ed=min(temp) 
        ED[row][slots] = min_ed
        perc_ED[row][slots] = (ED[row][slots]/diagonal_slot[row][slots])*100
        
        max_intrsct=max(intersect)  
        IoU[row][slots] = ((max_intrsct/area_slot[row][slots])*100)
        
        if perc_ED[row][slots] < 50:
            if IoU[row][slots] == 0:
                Status[row][slots] = 'VACANT'
            elif IoU[row][slots] > 0:
                Status[row][slots] = 'OCCUPIED'
        else:
            Status[row][slots] = 'VACANT'
        #draw ROI_slot
        mod = cv2.polylines(plottedImage, np.int32([ROI_slot[row][slots]]), True, (255,0,0), thickness=2)
        #draw Status_slot
        mod = cv2.putText(plottedImage, Status[row][slots], (int(centroid_slot_x[row][slots])-10, int(centroid_slot_y[row][slots])),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)

print(perc_ED)

[[1396.1224247657105, 1008.3403261352688, 955.4260527679222, 827.0915021905495, 651.5497429162116, 503.3997757657231, 428.0980890592822, 354.5530498649523, 187.8970289840907, 93.36761300249779, 25.786465156206734, 101.34025595740673, 34.139901578923656, 120.48751544998488, 144.83183741013727, 43.30522446256831, 141.71499164335464, 237.85884315764397], [1022.6145799882115, 741.242779213236, 644.5410838580761, 515.7618705028842, 436.20246566908133, 259.42637739546024, 192.97336192218955, 90.74443417043818, 24.040488433999446, 120.71742626203405, 188.90304801537175, 154.037779829102, 180.9865858859907, 229.13179379987918], [780.4785776342882, 619.2454929547304, 465.50745530196946, 388.51044641357726, 277.46735044649523, 171.2270882191707, 131.15760835618912, 109.85417555691122, 158.56936402811053, 196.17677114515666, 230.13940872854494, 263.3714675406771], [438.30476759307624, 343.10179826002314, 279.6200916855852, 224.2304378012147, 174.5293378737442, 200.51252795790222, 245.952083076076

In [219]:
print(IoU)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 82.0476858345021, 4.70542231491138, 85.39934492315444, 3.3073400173288796, 0.06415764449790817, 62.875978847048145, 1.139109791021262, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 69.3875802782054, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]


In [220]:
cv2.imshow('Shapes', mod)
cv2.waitKey(0)
cv2.destroyAllWindows()